In [1]:
# preliminary stuff - get demo BIDS dataset
!datalad install ///workshops/nipype-2017/ds000114
!datalad remove ds000114/derivatives/*
!datalad get -r ds000114
# also double up import to avoid ugly warnings printing during slideshow
from bids.layout import BIDSLayout
import os.path
import shutil
# house keeping
if os.path.exists("output"):
    shutil.rmtree("output", ignore_errors=True)

[WARNING] Running drop resulted in stderr output: git-annex: derivatives/* not found
git-annex: drop: 1 failed
 
drop(error): /imaging/jc01/bidstalk_2018/ds000114/derivatives/* (file) [not found]
[INFO   ] Installing <Dataset path=/imaging/jc01/bidstalk_2018/ds000114> recursively 
action summary:
  get (notneeded: 2)


Failed to import duecredit due to No module named 'duecredit'
/imaging/local/software/centos7/anaconda3/envs/cbu_nipy_1_00/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/imaging/local/software/centos7/anaconda3/envs/cbu_nipy_1_00/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/imaging/local/software/centos7/anaconda3/envs/cbu_nipy_1_00/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


In [2]:
# and demo dicoms for dcm2bids
!datalad install -g ///dicoms/dartmouth-phantoms/bids_test5-20170120

action summary:
  get (notneeded: 1)
  install (notneeded: 1)


# <center>Brain Imaging Data Structure (BIDS): A standard format for neuroscience data</center>
<center>Johan Carlin</center>
<p></p>
<center><a href>https://github.com/jooh/notebook_bids_OSD2018</a></center>
<center><img src="MRC_CBU_Cambridge_colour_web_A5.png"></center>

## In BIDS 1.1.1
* MRI - anatomical, functional, diffusion, field maps…
* MEG
* Physiological recordings
* Behavioural data

## Draft extensions
* Model specification
* Derivatives - structural, functional, resting state, diffusion…
* New modalities - EEG, PET, ASL, Eye tracking, intracranial EEG

# MRI example

In [3]:
# basic directory structure
!tree -CL 2 ds000114

ds000114
├── dataset_description.json
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── anat
│   ├── dwi
│   └── func
├── sub-02
│   ├── anat
│   ├── dwi
│   └── func
├── task-fingerfootlips_bold.json
├── task-fingerfootlips_events.tsv
└── task-linebisection_bold.json

8 directories, 6 files


In [4]:
# and inside one func folder
subdir = "ds000114/sub-01/func"
!ls -1 {subdir}
# and an events file
import pandas as pd
pd.read_csv(
    f"{subdir}/sub-01_task-linebisection_events.tsv",delimiter="\t")

sub-01_task-fingerfootlips_bold.nii.gz
sub-01_task-linebisection_bold.nii.gz
sub-01_task-linebisection_events.tsv


,onset,duration,weight,trial_type
0,24.3065,1,1.0,Incorrect_Task
1,25.9465,1,1.0,Correct_Task
2,27.5865,1,1.0,Correct_Task
3,29.2265,1,1.0,No_Response_Task
4,30.8664,1,1.0,Incorrect_Task
5,32.5064,1,1.0,No_Response_Task
6,34.1464,1,1.0,Correct_Task
7,35.7864,1,1.0,Incorrect_Task
8,37.4264,1,1.0,Correct_Task
9,39.0664,1,1.0,Incorrect_Task


# Why BIDS?

**Users** get easy access to new analysis tools, e.g.
`fmriprep ds000114 outdir sub-01 -w workdir`

In [5]:
# Developers:
from bids.layout import BIDSLayout
layout = BIDSLayout("ds000114")
subjects = layout.get_subjects()
print(subjects)
layout.get(subject=subjects[0], type="T1w", return_type='file')

['01', '02']


['/imaging/jc01/bidstalk_2018/ds000114/sub-01/anat/sub-01_T1w.nii.gz']

# Dcm2Bids
Fully automated conversion from dicom to BIDS-compliant output data.
* Converts dicoms to nifti (using dcm2niix), optionally anonymises structurals
* Moves niftis and header sidecars to the correct BIDS-format locations
* Adds custom fields to header sidecars as needed for BIDS compliance
* Initialises all required BIDS project files with sensible defaults

# Input dicoms

In [6]:
!tree -CL 1 bids_test5-20170120/phantom-1/

bids_test5-20170120/phantom-1/
├── 001-anat-scout_run+
├── 002-anat-scout_run+_MPR_sag
├── 003-anat-scout_run+_MPR_cor
├── 004-anat-scout_run+_MPR_tra
├── 005-anat_T1w_acq-MPRAGE_run+
├── 006-fmap_acq-2.4mm
├── 007-fmap_acq-2.4mm
└── 009-func_run+_task-rest_acq-2.4mm64sl1000tr32te600dyn

8 directories, 0 files


# Minimal configuration for a first test run
dcm2bids uses a configuration JSON file to map from dicom series to BIDS output files

In [7]:
# you could just type it out but we we will extend the dict below
import json
config = dict(descriptions=[])
def writeconfig(config, filename="config.json"):
    with open(filename, "w") as fp:
        json.dump(config, fp, indent=4)
writeconfig(config)
!cat config.json

{
    "descriptions": []
}

In [8]:
# First test run of dcm2bids
!dcm2bids -d bids_test5-20170120 -p 01 -c config.json -o output


'bids_test5-20170120_anat-scout_run+_20170120084340_series001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002a.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002b.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002c.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003a.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003b.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series004.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series

In [9]:
# Unsurprisingly there's not much in the output dir
# but NB the tmp_dcm2bids dir!
!tree -C output

output
├── dcm2bids.log
└── tmp_dcm2bids
    └── sub-01
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series001.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series001.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002a.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002a.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002b.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002b.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002c.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002c.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series003a.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series

In [10]:
# so the trick is to extend the config json to unambiguously identify our target
# acquisitions. Let's start with the t1. What's in the sidecar?
!cat output/tmp_dcm2bids/sub-01/bids_test5-20170120_anat_T1w_acq-MPRAGE_run+_20170120084340_series005.json

{
	"Modality": "MR",
	"MagneticFieldStrength": 3,
	"Manufacturer": "Siemens",
	"ManufacturersModelName": "Prisma",
	"InstitutionName": "Dartmouth_College_-_PBS",
	"InstitutionalDepartmentName": "Department",
	"InstitutionAddress": "Maynard_3_Hanover_NH_US_03755",
	"DeviceSerialNumber": "66112",
	"StationName": "AWP66112",
	"BodyPartExamined": "BRAIN",
	"PatientPosition": "HFS",
	"ProcedureStepDescription": "Heatherton_Stanger_1004_ontrack",
	"SoftwareVersions": "syngo_MR_E11",
	"MRAcquisitionType": "3D",
	"SeriesDescription": "anat_T1w_acq-MPRAGE_run+",
	"ProtocolName": "anat_T1w_acq-MPRAGE_run+",
	"ScanningSequence": "GR_IR",
	"SequenceVariant": "SK_SP_MP",
	"ScanOptions": "IR",
	"SequenceName": "_tfl3d1_16ns",
	"ImageType": ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
	"SeriesNumber": 5,
	"AcquisitionTime": "08:44:38.722500",
	"AcquisitionNumber": 1,
	"SliceThickness": 0.9,
	"SAR": 0.0381398,
	"EchoTime": 0.00232,
	"RepetitionTime": 2.3,
	"InversionTime": 0.933,
	"FlipAngle": 8,
	"Par

In [11]:
# so our first description might be
config["descriptions"] = []
config["descriptions"].append({"dataType": "anat", 
                               "suffix": "T1w",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": 
                                       "anat_T1w_acq-MPRAGE_run"
                                   }
                                   }
                               })
# save and print the config again
writeconfig(config)
!cat config.json

{
    "descriptions": [
        {
            "dataType": "anat",
            "suffix": "T1w",
            "criteria": {
                "in": {
                    "SeriesDescription": "anat_T1w_acq-MPRAGE_run"
                }
            }
        }
    ]
}

In [12]:
# take 2 - do we recognise the t1?
!dcm2bids -d bids_test5-20170120 -p 01 -c config.json -o output


'bids_test5-20170120_anat-scout_run+_20170120084340_series001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002a.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002b.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002c.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003a.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003b.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series004.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series

In [13]:
# now the output dir looks more interesting
!tree -C -I tmp_dcm2bids output

output
├── CHANGES
├── dataset_description.json
├── dcm2bids.log
├── derivatives
├── participants.tsv
├── README
└── sub-01
    └── anat
        ├── sub-01_T1w.json
        └── sub-01_T1w.nii.gz

3 directories, 7 files


In [14]:
# some of the required BIDS project root files have been initialised, e.g.
!cat output/dataset_description.json

{
    "Name": "",
    "BIDSVersion": "1.1.0",
    "License": "",
    "Authors": [
        ""
    ],
    "Acknowledgments": "",
    "HowToAcknowledge": "",
    "Funding": "",
    "ReferencesAndLinks": [
        ""
    ],
    "DatasetDOI": ""
}

# Scaling up to a complete dcm2bids conversion

In [15]:
# house keeping
del config["descriptions"][1:]
# we need a task for valid BIDS - "rest" is handy because it means we are exempt from
# needing the events.tsv file (phantoms are at rest by definition, right?)
config["descriptions"].append({"dataType": "func", 
                               "suffix": "bold",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "bold"
                                   }
                                   },
                               "customHeader": {"TaskName": "rest"}
                               })
# unfortunately no real way to distinguish the fieldmap magnitude images without
# matching on the echo times
echo1 = 0.00492
echo2 = 0.00738
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude1",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoTime": echo1
                                   }
                                   }
                               })
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude2",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoNumber": 2,
                                       "EchoTime": echo2
                                   }
                                   }
                               })
# need to intervene here to set EchoTime1 and EchoTime2 since it's a phase difference
# image (see https://github.com/rordenlab/dcm2niix/issues/139)
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "phasediff",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType": ["ORIGINAL", "PRIMARY", "P", "ND"]
                                   }
                                   },
                               "customHeader": {
                                   "EchoTime1": echo1,
                                   "EchoTime2": echo2
                               }
                               })
writeconfig(config, filename="config_complete.json")

In [16]:
# here's one I made earlier...
!cat config_complete.json

{
    "descriptions": [
        {
            "dataType": "anat",
            "suffix": "T1w",
            "criteria": {
                "in": {
                    "SeriesDescription": "anat_T1w_acq-MPRAGE_run"
                }
            }
        },
        {
            "dataType": "func",
            "suffix": "bold",
            "criteria": {
                "in": {
                    "PulseSequenceDetails": "bold"
                }
            },
            "customHeader": {
                "TaskName": "rest"
            }
        },
        {
            "dataType": "fmap",
            "suffix": "magnitude1",
            "criteria": {
                "in": {
                    "PulseSequenceDetails": "field_mapping"
                },
                "equal": {
                    "ImageType": [
                        "ORIGINAL",
                        "PRIMARY",
                        "M",
                        "ND",
                        "NORM"
                   

In [17]:
# take 3 - do we now recognise all the series we want to convert?
shutil.rmtree("output", ignore_errors=True)
!dcm2bids -d bids_test5-20170120 -p 01 -c config_complete.json -o output


'bids_test5-20170120_anat-scout_run+_20170120084340_series001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002a.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002b.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002c.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003a.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003b.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series004.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series

In [18]:
# the output directory looks fairly complete
!tree -C -I tmp_dcm2bids output

output
├── CHANGES
├── dataset_description.json
├── dcm2bids.log
├── derivatives
├── participants.tsv
├── README
└── sub-01
    ├── anat
    │   ├── sub-01_T1w.json
    │   └── sub-01_T1w.nii.gz
    ├── fmap
    │   ├── sub-01_magnitude1.json
    │   ├── sub-01_magnitude1.nii.gz
    │   ├── sub-01_magnitude2.json
    │   ├── sub-01_magnitude2.nii.gz
    │   ├── sub-01_phasediff.json
    │   └── sub-01_phasediff.nii.gz
    └── func
        ├── sub-01_task-rest_bold.json
        └── sub-01_task-rest_bold.nii.gz

5 directories, 15 files


# Checking conversions with bids-validator
Also available as a handy web app

In [21]:
# NB you would still need to add some more info manually in e.g. study_description.json
!bids-validator output

	1: Participants and phenotype .tsv files must have a 'participant_id' column. (code: 48 - PARTICIPANT_ID_COLUMN)
		./participants.tsv
			@ line: 1


        Summary:                 Available Tasks:        Available Modalities: 
        14 Files, 19.21MB        rest                    T1w                   
        1 - Subject                                      bold                  
        1 - Session                                      fieldmap              



# Resources

* [The BIDS starter kit](https://github.com/bids-standard/bids-starter-kit) - wiki with links to various resources
* [The official BIDS website](http://bids.neuroimaging.io/) - the official BIDS specification is surprisingly readable
* [Michael Notter's Nipype tutorial](https://github.com/miykael/nipype_tutorial) - great interactive tutorials on working with BIDS data in Python
* [BIDS-Validator web app](https://bids-standard.github.io/bids-validator/) - convenient way to check BIDS conversions
* [dcm2bids (jooh fork)](https://github.com/jooh/Dcm2Bids) - run the conversion code presented here

In [20]:
!cat output/participants.tsv